In [ ]:
import os
import sys
import json

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
from tqdm import tqdm

# 定义转换
transform = transforms.Compose([
    transforms.Resize((14,14)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 使用单个值进行归一化
])

# 加载 CIFAR-100 训练集
trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                         download=True, transform=transform)

# 定义想要的类别
filtered_classes = ['bear', 'tiger']

# 获取类别到索引的映射
class_to_idx = trainset.class_to_idx

# 筛选出对应索引
filtered_class_indices = [class_to_idx[cls] for cls in filtered_classes]

# 过滤数据集
filtered_trainset = [data for data in trainset if data[1] in filtered_class_indices]

# 过滤数据集并修改标签
filtered_trainset = []
for data, label in trainset:
    if label in filtered_class_indices:
        # 将 'bear' 的标签设置为 0，'tiger' 的标签设置为 1
        new_label = 0 if label == filtered_class_indices[0] else 1
        filtered_trainset.append((data, new_label))

# 划分训练集和验证集
train_size = int(0.8 * len(filtered_trainset))
validation_size = len(filtered_trainset) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(filtered_trainset, [train_size, validation_size])

print(train_size)
print(validation_size)
# 创建 DataLoader
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
validationloader = torch.utils.data.DataLoader(validation_dataset, batch_size=4, shuffle=False, num_workers=2)

train_num = train_size
val_num = validation_size

In [ ]:
print(filtered_class_indices)
print(trainloader)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 获取一个批次的数据
images, labels = next(iter(trainloader))

# 设置图像显示的行数和列数
rows = 2
cols = 2

# 创建一个图形和一组子图
fig, axes = plt.subplots(rows, cols, figsize=(10, 10))

# 遍历所有图像
for i in range(rows*cols):
    image = images[i].numpy() # 转换为 numpy 数组
    image = np.transpose(image, (1, 2, 0)) # 调整通道顺序
    print(image.shape)
    # 显示图像
    ax = axes[i//cols, i%cols]
    ax.imshow(image)
    ax.axis('off') # 关闭坐标轴

plt.show()


In [ ]:
# @title hybrid layer
# !pip install pennylane
!pip install pennylane==0.23.0
# !pip install pennylane
# !pip uninstall autoray
!pip install autoray==0.2.5
# !pip install pennylane-qulacs
import torch
import torch.nn as nn
import numpy as np
import pennylane as qml
from math import ceil
from math import pi

torch.manual_seed(0)

n_qubits = 4
n_layers = 1
n_class = 2
n_features = 196
image_x_y_dim = 14
kernel_size = n_qubits
stride = 2

dev = qml.device("default.qubit", wires=n_qubits)


def circuit(inputs, weights):
    var_per_qubit = int(len(inputs) / n_qubits) + 1
    encoding_gates = ['RZ', 'RY'] * ceil(var_per_qubit / 2)
    for qub in range(n_qubits):
        qml.Hadamard(wires=qub)
        for i in range(var_per_qubit):
            if (qub * var_per_qubit + i) < len(inputs):
                exec('qml.{}({}, wires = {})'.format(encoding_gates[i], inputs[qub * var_per_qubit + i], qub))
            else:  # load nothing
                pass

    for l in range(n_layers):
        for i in range(n_qubits):
            qml.CRZ(weights[l, i], wires=[i, (i + 1) % n_qubits])
            # qml.CNOT(wires = [i, (i + 1) % n_qubits])
        for j in range(n_qubits, 2 * n_qubits):
            qml.RY(weights[l, j], wires=j % n_qubits)

    _expectations = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    return _expectations
    # return qml.expval(qml.PauliZ(0))


class Quanv2d(nn.Module):
    def __init__(self, kernel_size=None, stride=None):
        super(Quanv2d, self).__init__()
        weight_shapes = {"weights": (n_layers, 2 * n_qubits)}
        qnode = qml.QNode(circuit, dev, interface='torch', diff_method='best')
        self.ql1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, X):
        assert len(X.shape) == 4
        bs = X.shape[0]
        XL = []
        for i in range(0, X.shape[2] - 2, stride):
            for j in range(0, X.shape[3] - 2, stride):
                XL.append(self.ql1(torch.flatten(X[:, :, i:i + kernel_size, j:j + kernel_size], start_dim=1)))
        X = torch.cat(XL, dim=1).view(bs,4,6,6)
        return X


class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        self.ql1 = Quanv2d(kernel_size=kernel_size, stride=stride)
        self.conv1 = nn.Conv2d(4,16,3,stride=1)
        self.fc1 = nn.Linear(16*4*4, n_class * 2)
        self.lr1 = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(n_class * 2, n_class)

    def forward(self, X):
        bs = X.shape[0]
        X = X.view(bs, 1, image_x_y_dim, image_x_y_dim)
        X = self.ql1(X)
        X = self.lr1(self.conv1(X))
        X = X.view(bs,-1)
        X = self.fc1(X)
        X = self.lr1(X)
        X = self.fc2(X)
        return X


In [ ]:



import torch.nn.functional as F
kernel_size=2


def circuit(inputs, weights):
    var_per_qubit = int(len(inputs) / n_qubits) + 1
    encoding_gates = ['RZ', 'RY'] * ceil(var_per_qubit / 2)

    for qub in range(n_qubits):
        qml.Hadamard(wires=qub)
        for i in range(var_per_qubit):
            idx = qub * var_per_qubit + i
            if idx < len(inputs):
                if encoding_gates[i] == 'RZ':
                    qml.RZ(inputs[idx], wires=qub)
                elif encoding_gates[i] == 'RY':
                    qml.RY(inputs[idx], wires=qub)

    for l in range(n_layers):
        for i in range(n_qubits):
            qml.CRZ(weights[l, i], wires=[i, (i + 1) % n_qubits])
        for j in range(n_qubits, 2 * n_qubits):
            qml.RY(weights[l, j], wires=j % n_qubits)

    _expectations = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    return _expectations

class Quanv2d(nn.Module):
    def __init__(self, kernel_size=None, stride=None):
        super(Quanv2d, self).__init__()
        weight_shapes = {"weights": (n_layers, 2 * n_qubits)}
        qnode = qml.QNode(circuit, dev, interface='torch', diff_method='best')
        self.ql1 = qml.qnn.TorchLayer(qnode, weight_shapes)
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, X):
        assert len(X.shape) == 4
        bs = X.shape[0]
        XL = []
        for i in range(0, X.shape[2] - 2, stride):
            for j in range(0, X.shape[3] - 2, stride):
                XL.append(self.ql1(torch.flatten(X[:, :, i:i + kernel_size, j:j + kernel_size], start_dim=1)))
        X = torch.cat(XL, dim=1).view(bs,4,6,6)
        return X


class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        self.ql1 = Quanv2d(kernel_size=kernel_size, stride=stride)
        self.conv1 = nn.Conv2d(4,16,3,stride=1)
        self.fc1 = nn.Linear(16*4*4, n_class * 2)
        self.lr1 = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(n_class * 2, n_class)

    def forward(self, X):
        bs = X.shape[0]
        X = X.view(bs, 1, image_x_y_dim, image_x_y_dim)
        X = self.ql1(X) #output(bs, 4, 6, 6)
        X = self.lr1(self.conv1(X))
        X = X.view(bs,-1)
        X = self.fc1(X)
        X = self.lr1(X)
        X = self.fc2(X)
        return X


def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using {} device.".format(device))

#




    batch_size = 20
    train_num = len(trainloader)
    # val_num = len(validate_dataset)


    print("using {} images for training, {} images for validation.".format(train_num,
                                                                           val_num))



    net = Net()
    # net =SimpleNet()
    net.to(device)
    loss_function = nn.CrossEntropyLoss()
    # pata = list(net.parameters())
    optimizer = optim.Adam(net.parameters(), lr=0.0002)

    epochs = 5
    # save_path = './AlexNet.pth'
    best_acc = 0.0
    train_steps = len(trainloader)

    for epoch in range(epochs):
        # train
        net.train()
        running_loss = 0.0
        train_bar = tqdm(trainloader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            print(images.shape)
            print(labels)
            optimizer.zero_grad()
            outputs = net(images.to(device))
            loss = loss_function(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)

        # validate
        net.eval()
        acc = 0.0  # accumulate accurate number / epoch
        with torch.no_grad():
            val_bar = tqdm(validationloader, file=sys.stdout)
            for val_data in val_bar:
                val_images, val_labels = val_data
                outputs = net(val_images.to(device))
                predict_y = torch.max(outputs, dim=1)[1]
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

        val_accurate = acc / val_num
        print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f' %
              (epoch + 1, running_loss / train_steps, val_accurate))

        # if val_accurate > best_acc:
        #     best_acc = val_accurate
        #     torch.save(net.state_dict(), save_path)

    print('Finished Training')


if __name__ == '__main__':
    main()